<a href="https://colab.research.google.com/github/Procatv/USCEats/blob/main/Copy_of_project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3, Due: Apr. 30, 2025
**CSCI-360 Introduction to Artificial Intelligence (2025 Spring)**

## Instructions
In the following Python code blocks, write your code in the section marked with
```python
###
### YOUR CODE HERE
###
```
and
```python
### BEGIN HELPER

### END HELPER
```
Specifically, the `SOLUTION` sections will be graded with test cases; you can leave `HELPER` sections blank if no need.
Do not modify other code block. If you have any question, please ask on Piazza or go to an office hour.

In [ ]:
# example
! pip install numpy

### BEGIN HELPER

### END HELPER

## Skeleton

In [ ]:
# Predefined Constants
gamma = 0.9
epsilon = 0.01

# Specifying variable types.
from typing import List, Tuple, Dict
# We will be using a tuple (x,y) to represent a state,
# e.g., (0, 0) means the robot is currently at (0, 0) on the map
# We will be using a tuple (delta_x,delta_y) to represent an action,
# e.g., (1, 0) means the going east (x_new=x+1, y_new=y)
State = Tuple[int, int]
Action = Tuple[int, int]

In [ ]:
class GridMDP:

    def __init__(self, grid_size: int, gamma: float,
                 obstacles: List[State], terminal: State):

        self.grid_size = grid_size
        self.gamma = gamma
        self.obstacles = obstacles
        self.terminal = terminal

        self.actions = NORTH, SOUTH, EAST, WEST = [(0, -1), (0, 1), (1, 0), (-1, 0)]

        self.states = set((x, y) for x in range(grid_size) for y in range(grid_size))
        self.rewards = self.create_reward_function()
        self.transitions = self.create_transition_function()

    def go(self, current_state: State, action: Action) -> State:
        """Given the current position and action (one of {EAST, NORTH, WEST, SOUTH}),
        Compute the next position. Pay attention to cases where your robot goes off the site."""
        ###
        x, y = current_state
        dx, dy = action
        new_state = (x + dx, y + dy)
        if new_state in self.states:
            return new_state
        else:
            return current_state
        ###

    def create_reward_function(self) -> Dict[State, int]:
        """Create a reward function R given the obstacles and terminal information,
        Note: R is a dictionary. R[state] = reward."""
        ###
        R: Dict[State, int] = {}
        for s in self.states:
          if s == self.terminal:
            R[s] = 100
          elif s in self.obstacles:
            R[s] = -100
          else:
            R[s] = -1
        return R
        ###

    def create_transition_function(self) -> Dict[State, Dict[Action, List[Tuple[float, State]]]]:
        """Create a transition function T, in the following format:
        T[state][action] = [(probability 1, possible next state 1),
                            (probability 2, possible next state 2),
                            ...]
        Note: T is a dictionary. T[state] is a dictionary.
        T[state][action] is a list."""
        ###
        T: Dict[State, Dict[Action, List[Tuple[float, State]]]] = {}
        for s in self.states:
          T[s] = {}
          for a in self.actions:
            outcomes: List[Tuple[float, State]] = []
            for a2 in self.actions:
              p = 0.7 if a2 == a else 0.1
              s2 = self.go(s, a2)
              outcomes.append((p, s2))
            T[s][a] = outcomes
        return T
        ###


In [ ]:
class GridMDPSolver():
    def __init__(self, grid_mdp: GridMDP):
        self.grid_mdp = grid_mdp

    def solve(self, epsilon):
        U = self.value_iteration(epsilon)
        best_policy = self.best_policy(U)
        display_string = self.render_arrows(best_policy)
        return display_string

    def expected_utility(self, state: State, action: Action, utility: Dict[State, float]) -> float:
        """Given a state, an action, a utility function.
        Return the expected utility."""

        T = self.grid_mdp.transitions
        expected_utility = 0.0

        ###
        for p, next_s in T[state][action]:
          expected_utility += p * utility[next_s]
        ###

        return expected_utility

    def value_iteration(self, epsilon) -> Dict[State, float]:
        """Implement the value iteration algorithm. Return a utility dictionary U.
        You may want to use the `expected_utility` function defined above."""

        U_prime = {state: 0 for state in self.grid_mdp.states}
        U = U_prime.copy()

        gamma = self.grid_mdp.gamma
        states = self.grid_mdp.states
        actions = self.grid_mdp.actions
        R = self.grid_mdp.rewards

        ###
        threshold = epsilon * (1 - gamma) / gamma
        while True:
          delta = 0.0
          for s in states:
            if s == self.grid_mdp.terminal:
              U_prime[s] = R[s]
            else:
              best_succ = max(self.expected_utility(s, a, U) for a in actions)
              U_prime[s] = R[s] + gamma * best_succ
            delta = max(delta, abs(U_prime[s] - U[s]))
          U = U_prime.copy()
          if delta < threshold:
            break
        ###

        return U


    def best_policy(self, utility: Dict[State, float]) -> Dict[State, Action]:
        """Create a policy by using the principle of maximum expected utility.
        You may want to use the `expected_utility` function defined above.
        """

        states = self.grid_mdp.states
        actions = self.grid_mdp.actions
        p_best = {}

        ###
        for s in states:
          if s == self.grid_mdp.terminal:
            continue
          best_a = actions[0]
          best_val = self.expected_utility(s, best_a, utility)
          for a in actions[1:]:
            val = self.expected_utility(s, a, utility)
            if val > best_val:
              best_a = a
              best_val = val
          p_best[s] = best_a
        ###

        return p_best


    def render_arrows(self, policy: Dict[State, Action]) -> str:
        """Format the policy into a string containing `<^>v*o` for display.
        Example policy: (0,0) -> NORTH, (0,1) -> NORTH,
                        (1,0) -> SOUTH, (1,1) -> WEST.
        Example display string: `^v\n^<`.

        Note 1: In the grid, we use (col, row) coordinates.
        The display string should reflect this.

        Note 2: Obstacles should be represented with `o`.
        Terminal should be represented with `.`.
        """
        states = self.grid_mdp.states
        grid_size = self.grid_mdp.grid_size
        NORTH, SOUTH, EAST, WEST = [(0, -1), (0, 1), (1, 0), (-1, 0)]
        display_string = ""

        ###
        for y in range(grid_size):
          for x in range(grid_size):
            s = (x,y)
            if s == self.grid_mdp.terminal:
              display_string += '.'
            elif s in self.grid_mdp.obstacles:
              display_string += 'o'
            else:
              a = policy[s]
              if a == NORTH:
                display_string += '^'
              elif a == SOUTH:
                display_string += 'v'
              elif a == EAST:
                display_string += '>'
              elif a == WEST:
                display_string += '<'
          if y < grid_size - 1:
            display_string += '\n'
        ###

        return display_string


## Revealed Test Cases

In [ ]:
def evaluate(test_case, ground_truth_output, your_output):
    your_output = your_output.strip()
    assert len(ground_truth_output) == len(your_output)
    total_count, correct_count = 0, 0
    for a, b in zip(ground_truth_output, your_output):
        if a == "\n": continue
        total_count += 1
        correct_count += int(a==b)
    acc = correct_count / total_count
    print(f"{test_case}, acc={correct_count}/{total_count}={acc:.4f}")
    assert acc >= 0.90 or (total_count - correct_count <= 4), \
        f"test name {test_case}, your output matches with {acc:.4f} of the groundtruth answer."

In [ ]:
test_cases = {
    "revealed_test_0": [4, 2, [(0, 0), (3, 0)], (2, 2), "ovvo\nvvvv\n>>.<\n>>^<"],
    "revealed_test_1": [4, 3, [(1, 3), (3, 3), (3, 0)], (1, 0), ">.<o\n^^<<\n^^^<\n^o^o"],
    'revealed_test_2': [4, 1, [(3, 2)], (2, 1), '>>v<\n>>.<\n^^^o\n^^^<'],
    "revealed_test_3": [8, 7, [(4, 6), (2, 7), (1, 5), (2, 0), (7, 5), (3, 4), (5, 3)], (0, 7), 'v<ovv<<<\nvvvv<<<<\nv<<<<<^^\nv<<<^o^^\nv<<o^v^^\nvov<<<vo\nv<<<ovvv\n.<o^<>v<'],
    "revealed_test_4": [12, 24, [(2, 2), (10, 3), (6, 8), (4, 1), (1, 4), (5, 6), (11, 8), (11, 1), (1, 6), (9, 8), (8, 11), (7, 1), (6, 4), (1, 0), (3, 4), (3, 6), (11, 6), (10, 11), (0, 2), (3, 8), (1, 10), (8, 3), (7, 6), (6, 11)],(9, 6), 'vo>>>>>>>v<<\n>>>^ovvo>v<o\novo>>vvv>v<v\nv>>>>>>vovov\nvovovvo>vvv<\nv<>>>>>>>v<<\nvovo^o^o>.<o\nv<v>>>>>^^^<\n<<<ovvo>^o^o\n^<>>>>>>^<^<\n^o>>>^>^^^^^\nv>>>^^o^o^o^'],
}

for k, v in test_cases.items():
    grid_size, n_obstacles, obstacles, terminal, ground_truth_output = v
    grid_mdp = GridMDP(grid_size, gamma, obstacles, terminal)
    your_output = GridMDPSolver(grid_mdp).solve(epsilon)
    evaluate(k, ground_truth_output, your_output)

revealed_test_0, acc=16/16=1.0000
revealed_test_1, acc=16/16=1.0000
revealed_test_2, acc=16/16=1.0000
revealed_test_3, acc=64/64=1.0000
revealed_test_4, acc=144/144=1.0000


## Hidden Test Cases

In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
